In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc, classification_report
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multiple-disease-prediction/Blood_samples_dataset_balanced_2(f).csv
/kaggle/input/multiple-disease-prediction/blood_samples_dataset_test.csv


## Reading Dataset

In [3]:
BASE_DIR = "/kaggle/input/multiple-disease-prediction/"
df = pd.read_csv(BASE_DIR+"Blood_samples_dataset_balanced_2(f).csv")
df.head()
df

,Glucose,Cholesterol,Hemoglobin,Platelets,White Blood Cells,Red Blood Cells,Hematocrit,Mean Corpuscular Volume,Mean Corpuscular Hemoglobin,Mean Corpuscular Hemoglobin Concentration,...,HbA1c,LDL Cholesterol,HDL Cholesterol,ALT,AST,Heart Rate,Creatinine,Troponin,C-reactive Protein,Disease
0,0.739597,0.650198,0.713631,0.868491,0.687433,0.529895,0.290006,0.631045,0.001328,0.795829,...,0.502665,0.215560,0.512941,0.064187,0.610827,0.939485,0.095512,0.465957,0.769230,Healthy
1,0.121786,0.023058,0.944893,0.905372,0.507711,0.403033,0.164216,0.307553,0.207938,0.505562,...,0.856810,0.652465,0.106961,0.942549,0.344261,0.666368,0.659060,0.816982,0.401166,Diabetes
2,0.452539,0.116135,0.544560,0.400640,0.294538,0.382021,0.625267,0.295122,0.868369,0.026808,...,0.466795,0.387332,0.421763,0.007186,0.506918,0.431704,0.417295,0.799074,0.779208,Thalasse
3,0.136609,0.015605,0.419957,0.191487,0.081168,0.166214,0.073293,0.668719,0.125447,0.501051,...,0.016256,0.040137,0.826721,0.265415,0.594148,0.225756,0.490349,0.637061,0.354094,Anemia
4,0.176737,0.752220,0.971779,0.785286,0.443880,0.439851,0.894991,0.442159,0.257288,0.805987,...,0.429431,0.146294,0.221574,0.015280,0.567115,0.841412,0.153350,0.794008,0.094970,Thalasse
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,0.012956,0.336925,0.451218,0.175006,0.734664,0.382770,0.656463,0.177502,0.808162,0.684499,...,0.670665,0.311568,0.595083,0.155866,0.885812,0.636125,0.132226,0.716519,0.006121,Diabetes
2347,0.407101,0.124738,0.983306,0.663867,0.361113,0.663716,0.232516,0.341056,0.847441,0.309766,...,0.491185,0.701914,0.218104,0.790341,0.570902,0.339125,0.310964,0.310900,0.622403,Thalasse
2348,0.344356,0.783918,0.582171,0.996841,0.065363,0.242885,0.658851,0.543017,0.290106,0.838722,...,0.141738,0.155871,0.473638,0.250535,0.387197,0.344728,0.606719,0.395145,0.134021,Anemia
2349,0.351722,0.014278,0.898615,0.167550,0.727148,0.046091,0.900434,0.136227,0.134361,0.279219,...,0.570553,0.171245,0.858352,0.362012,0.290984,0.996873,0.882164,0.411158,0.146255,Diabetes


## Pre-Processing of Dataset

In [ ]:
df.dtypes

In [ ]:
df.Disease.value_counts()

In [ ]:
df.isna().sum()

In [ ]:
df.Disease = df.Disease.map({
    "Anemia":0,
    "Healthy":1,
    "Diabetes":2,
    "Thalasse":3,
    "Thromboc":4
})

In [ ]:
df.dtypes

## Dividing Dataset into X and y labels

In [ ]:
X = df.drop(["Disease"], axis=1)
y = df.Disease

## Dividing into training and testing data

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X,y,test_size=0.3, random_state=0)

## Logistic Regression

In [ ]:
model = LogisticRegression()
model.fit(xtrain, ytrain)

In [ ]:
model.score(xtest, ytest)

In [ ]:
preds = model.predict(xtest)
cm = confusion_matrix(ytest, preds)
cmdf = pd.DataFrame(cm, index=["Anemia","Healthy","Diabetes","Thalasse","Thromboc"], columns=["Anemia","Healthy","Diabetes","Thalasse","Thromboc"])
plt.figure(figsize=(8,6))
sns.heatmap(cmdf)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
cm

## Decision Tree Classifier

In [ ]:
model = DecisionTreeClassifier()
model.fit(xtrain, ytrain)

In [ ]:
model.score(xtest, ytest)

In [ ]:
preds = model.predict(xtest)
cm = confusion_matrix(ytest, preds)
cmdf = pd.DataFrame(cm, index=["Anemia","Healthy","Diabetes","Thalasse","Thromboc"], columns=["Anemia","Healthy","Diabetes","Thalasse","Thromboc"])
plt.figure(figsize=(8,6))
sns.heatmap(cmdf)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

Similarly we can apply other Classifiers for the model. Since, we are achieving 100% accuracy (accuracy score of 1.0), I will not be going further with other classifiers or hyperparameter tuning.

We will see hyperparameter tuning in some other dataset where we would be required to optimize our algorithm for accuracy